In [1]:
# Esta es una modificacion del script original para google colab, para que funcione en github codespaces
# Hecho por @LUCAST o @Lucast1574 o @Lucast_1574 en github y otras redes sociales
# Solo versiones +1.17
# Versiones disponibles testeadas:
# - 1.20.4
# - 1.20.3
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.4
# - 1.19.3
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - Para versiones antiguas, ya esta disponible a exepcion de forge, seguir los pasos para usar un forge antiguo



version = '1.20.1' # Version de Minecraft a usar

#Chose server type. Currently available versions:
# - paper (Lo más vanilla. Es lo mismo que SPIGOT, bukkit, etc)
# - forge (Para usar MODS. Es básicamente FORGE) [Aviso, la instalación tarda APROX 10-15 min, al ejecutar el primer paso, esperar a que ponga COMPLETADO]
# - oldforge (Para usar MODS. Es básicamente FORGE) [Aviso, la instalación tarda APROX 10-15 min, al ejecutar el primer paso, esperar a que ponga COMPLETADO] Solo para versiones antiguas -1.17
# - fabric (Para jugar Fabric. Es una alternativa a Forge)
# - vanilla (Minecraft Vanilla sin mods ni plugins)
# - snapshot (En "version" tendrás que poner el nombre de la SNAPSHOT, EJ: 23w42a)
# - mohist (Versión de Minecraft que combina Forge y Bukkit) 1.20.1 29/09/24 ultima version disponible
# - catserver (Versión de Minecraft que combina Forge y Bukkit) 1.18.2 29/09/24 ultima version disponible
# - purpur (Versión de Minecraft que combina paper y Bukkit)

# Para usar paper primero tienes que instalar el vanilla
# Para usar mohist primero tienes que instalar el forge
# Para usar catserver primero tienes que instalar el forge
# Para usar purpur primero tienes que instalar el fabric

server_type = 'forge'

#Importa librerias necesarias para el funcionamiento del script 

import os
import requests
import json


server_directory = "Minecraft-server"

# Crea el directorio del servidor en codespace

os.makedirs(server_directory, exist_ok=True)
os.chdir(server_directory)

# Inicio interno... Descarga el servidor 

if server_type == 'paper':
    a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)

    b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))

    print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])
    serverURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

elif server_type == 'forge':
    serverURL = "https://serverjars.com/api/fetchJar/modded/forge/" + version

elif server_type == 'vanilla':
    serverURL = "https://serverjars.com/api/fetchJar/vanilla/vanilla/" + version

elif server_type == 'snapshot':
    serverURL = "https://serverjars.com/api/fetchJar/vanilla/snapshot/" + version

elif server_type == 'fabric':
    serverURL = 'https://serverjars.com/api/fetchJar/modded/fabric/' + version

elif server_type == 'mohist':
    serverURL = "https://serverjars.com/api/fetchJar/modded/mohist/" + version
    
elif server_type == 'catserver':
    serverURL = "https://serverjars.com/api/fetchJar/modded/catserver/" + version

elif server_type == 'purpur':
    serverURL = "https://serverjars.com/api/fetchJar/servers/purpur/" + version

# Lista de nombres de los archivos

jar_name = {'paper': 'paper.jar', 'fabric': 'fabric-server-launch.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': "snapshot.jar", 'mohist': "mohist.jar", 
            'catserver': "catserver.jar", 'purpur': "purpur.jar"}

print('Descargando el tipo de servidor: ')

if serverURL:
    print('Descargando a GitHub Codespaces...')
    r = requests.get(serverURL)

    if r.status_code == 200:
        with open(jar_name[server_type], 'wb') as f:
            f.write(r.content)
    else:
        print('Error '+ str(r.status_code) + '! La versión que has elegido probablemente no funciona.')


# Correr en un camino especifico

if server_type == 'fabric':
    !java -jar fabric-server-launch.jar server -mcversion $version -downloadMinecraft

if server_type == 'forge':
    %cd "/workspaces/codespaces-jupyter/Minecraft-server"
    !java -jar forge.jar --installServer

if server_type == 'vanilla':
    %cd "/workspaces/codespaces-jupyter/Minecraft-server"
    !java -jar vanilla.jar --installServer

# Guarda el archivo de configuración localmente
colabconfig = {"server_type": server_type, "server_version": version}
with open("colabconfig.json", 'w') as config_file:
    json.dump(colabconfig, config_file)

# Imprime un mensaje de finalización
print('Completado!')
# Cambie el eula a true para aceptar los términos de Minecraft
!echo "eula=true" >> eula.txt


In [2]:
#Tienes que poner el spawn proteccion en 0 para que puedas jugar de manera normal(esto dentro de Server.properties)
#importa librerias necesarias para ejecucion

import requests

import time

import os

import re

import json

import glob

import threading

import subprocess


#Actualizacion de lista de paquetes

!sudo apt update &>/dev/null && echo "el apt cache fue actualizado correctamente" || echo "la actualizacion del apt cache fallo, es posible que recibas paquetes obsoletos"

#Asignar archivos al directorio


os.makedirs("/workspaces/codespaces-jupyter/Minecraft-server", exist_ok=True)

os.chdir("/workspaces/codespaces-jupyter/Minecraft-server")

!ls #lista los archivos del directorio para verificarlos



#Importacion del archivo de configuracion


config_path = "/workspaces/codespaces-jupyter/Minecraft-server/colabconfig.json"

if os.path.isfile(config_path):

    with open(config_path) as config_file:

        colabconfig = json.load(config_file)

else:

    colabconfig = {"server_type": "generic"} # Usar la configura default si no existe

    with open(config_path, 'w') as new_config_file:

        json.dump(colabconfig, new_config_file)

# Instalar OpenJDK

version = colabconfig["server_version"]

# Ejecuta un comando en la terminal

def installjava17(comando):
    proceso = subprocess.run(comando, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Comprueba si el comando se ejecutó con éxito
    if proceso.returncode == 0:
        print("OpenJDK 17 instalado correctamente")
    else:
        print("Fallo al instalar OpenJDK 17")

def installjava11(comando):
    proceso = subprocess.run(comando, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Comprueba si el comando se ejecutó con éxito
    if proceso.returncode == 0:
        print("OpenJDK 11 instalado correctamente")
    else:
        print("Fallo al instalar OpenJDK 11")

def installjava8(comando):
    proceso = subprocess.run(comando, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Comprueba si el comando se ejecutó con éxito
    if proceso.returncode == 0:
        print("OpenJDK 8 instalado correctamente")
    else:
        print("Fallo al instalar OpenJDK 8")
        
def purgejava(comando):
    proceso = subprocess.run(comando, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Comprueba si el comando se ejecutó con éxito
    if proceso.returncode == 0:
        print("OpenJDK purgado correctamente")
    else:
        print("Fallo al purgar OpenJDK")

# Instala OpenJDK 17 si la versión de Minecraft es 1.17 o superior
# Si la versión es inferior, instala OpenJDK 8

if version < "1.14":
    # Purga versiones anteriores de OpenJDK
    purgejava('sudo apt-get purge openjdk* -y')
    # Instala OpenJDK 8
    installjava8('sudo apt-get install openjdk-8-jre-headless -y')

elif version >= "1.14" and version < "1.17":
    # Purga versiones anteriores de OpenJDK
    purgejava('sudo apt-get purge openjdk* -y')
    # Instala OpenJDK 11
    installjava11('sudo apt-get install openjdk-11-jre-headless -y')

else:
    # Purga versiones anteriores de OpenJDK
    purgejava('sudo apt-get purge openjdk* -y')
    # Instala OpenJDK 17
    installjava17('sudo apt-get install openjdk-17-jre-headless -y')

# Chequeo de version de java

java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

#Lista de posibles archivos .jar

jar_list = {'paper': 'paper.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar', 'mohist': "mohist.jar", 'catserver': "catserver.jar", 'purpur': "purpur.jar"}

jar_name = jar_list[colabconfig["server_type"]]

# Argumentos de java.

if colabconfig["server_type"] == "paper":

    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"

else:

    server_flags = "" # No se necesitan banderas para servidores que no sean de paper

#Argumentos JVM

memory_allocation = "-Xms10G -Xmx10G"



# Elige el servicio de tunelacion que desees

# Opciones disponibles: ngrok

tunnel_service = "playit"

print("Usando", tunnel_service)


#Condicional de tunelacion del servidor



if tunnel_service == "ngrok":

    !pip -q install pyngrok  #Aqui se instala el ngrok en el servidor

    from pyngrok import conf, ngrok



    # Obten tu token de ngrok

    print("Consigue tu authtoken de https://dashboard.ngrok.com/auth") #Consigue aqui tu token de ngrok, sera dinamico, si quieres que sea estatico, necesitaras la version premiun de ngrok

    import getpass



    # v - - - - - - - TOKEN - - - - - - - v



    authtoken = "TU TOKEN AQUI" # <---- TOKEN NGROK (PON AQUÍ TU TOKEN [TIENE QUE ESTAR ENTRE LAS "COMILLAS"])

    ! ngrok authtoken $authtoken # logeo en ngrok



    #Establecimiento de region de ngrok



    #v - - - - - - - REGIONES DISPONIBLES - - - - - - - v

    # ap - Asia/Pacifico (Singapore)

    # au - Australia (Sydney)

    # eu - Europa (Frankfurt - Alemania)

    # in - India (Mumbai)

    # jp - Japon (Tokyo)

    # sa - America del sur (São Paulo - Brasil)

    # us - Estados unidos (Ohio)

    conf.get_default().region = 'us' # <--- Cambia esto por la region que quieras



    # Conectar a ngrok

    url = ngrok.connect(25565, 'tcp')

    print('La IP de tu servidor es ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

    #Si obtienes el servicio premiun de ngrok borra las dos lineas de codigo de arriba y quitale el # a estas a continuacion:

    #subdominio = "mi-subdominio"

    #url = ngrok.connect(25565, 'tcp', subdomain=subdominio)

    #print('La IP de tu servidor es ' + url.replace('tcp://', ''))


elif tunnel_service == "playit":

    subprocess.run(["curl", "-SsL", "https://playit-cloud.github.io/ppa/key.gpg", "-o", "key.gpg"], check=True)
    subprocess.run(["sudo", "apt-key", "add", "key.gpg"], check=True)

    # Añadiendo el repositorio de Playit.gg
    subprocess.run(["sudo", "curl", "-SsL", "-o", "/etc/apt/sources.list.d/playit-cloud.list", "https://playit-cloud.github.io/ppa/playit-cloud.list"], check=True)

    # Actualizando la lista de paquetes
    subprocess.run(["sudo", "apt", "upgrade","-y"], check=True)

    # Instalando Playit.gg
    print("Installing playit ...")
    result = subprocess.run(["sudo", "apt", "install", "playit","-y"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Comprueba si la instalación fue exitosa
    if result.returncode == 0:
        print("Playit.gg installed")
    else:
        print("Failed to install Playit.gg")

    # Iniciando el servidor con Playit.gg
    print('Starting server...')
    result = subprocess.Popen(["playit"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode == 0:
        print("Initialized playit")
    else:
        print("Failed at initialized server")
    #subprocess.run(["java", "$memory_allocation", "$server_flags", "-jar", "$jar_name", "nogui"],check=True)

    # Obteniendo la IP del servidor
    print ("Charging status...")
    result = subprocess.run(["playit", "status"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode == 0:
        print("Tu ip es: " + result.stdout.decode())
    else:
        print("Failed to obtain IP")
    

#Obtiene la lista de usuarios de JSONPlaceholder
#Para mantener el servidor activo
def get_users():
    print("Obteniendo usuarios")
    url = 'https://jsonplaceholder.typicode.com/users'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print("Lista de usuarios:")
            for user in data:
                print(f"Nombre: {user['name']}, Email: {user['email']}")
        else:
            print(f"Error al hacer la solicitud. Código de estado: {response.status_code}")
    except requests.RequestException as e:
        print(f"Error de conexión: {e}")

# Función para ejecutar get_users() cada 25 minutos
def repeat_get_users():
    while True:
        get_users()
        time.sleep(600)  # Espera 10 minutos

# Inicia un hilo para ejecutar repeat_get_users() en segundo plano
user_thread = threading.Thread(target=repeat_get_users)
user_thread.start()

#Obtener el archivo de configuracion
stype = colabconfig["server_type"]
version = colabconfig["server_version"]

#Imprimir el tipo de servidor y la version
print("Estas jugando a minecraft: " + stype + " en su version: "+ version)

#Si el servidor es de tipo Forge antiguo no va a funcionar
if colabconfig["server_type"] == "forge" and colabconfig["server_version"] < "1.17.1":
    print("La version de Forge no es compatible con este script. Por favor, usa una version de Forge 1.17.1 o superior.")


#Si el servidor es de tipo Forge encontrara los archivos necesarios para ejecutar el servidor

if colabconfig["server_type"] == "forge":

    # Obtiene la versión de Forge
    
    forgefiles = f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/"
    
    forgepreversion = os.listdir(forgefiles)
        
    if forgepreversion:
        
        forgeversion = forgepreversion[0]
        forgeversionchecked = forgeversion.replace(".jar", "")
        print("La versión de Forge es:", forgeversionchecked)
        
    else:
        
        print("No se encontraron archivos en el directorio Forge.")
        
    pathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/{forgeversionchecked}/unix_args.txt")

    if pathtoforge: # Checa si la lista no esta vacia

        path = pathtoforge[0] # Obtiene la primera ruta de la lista

        print(path)

        !java @user_jvm_args.txt "@{path}" "$@"

    else:

        print("No se encontro el archivo unix_args.txt.")

#Si el servidor es de tipo Vanilla o paper o mohist o catserver o fabric o purpur

else:

    #Ejecuta el archivo jar del servidor
    
    !java $memory_allocation $server_flags -jar $jar_name nogui

# Ciclo para realizar las solicitudes a JSONPlaceholder cada minuto

while True:
    time.sleep(60)  # Espera 1 minuto

el apt cache fue actualizado correctamente
banned-ips.json      forge.jar	    ops.json	       usernamecache.json
banned-players.json  forge.jar.log  patchouli_books    whitelist.json
colabconfig.json     key.gpg	    run.bat	       world
config		     libraries	    run.sh	       xaerominimap-common.txt
defaultconfigs	     logs	    scripts	       xaeroworldmap-common.txt
emotes		     mod_data	    server.properties
essential	     modernfix	    user_jvm_args.txt
eula.txt	     mods	    usercache.json
OpenJDK purgado correctamente
OpenJDK 17 instalado correctamente
Usando playit


OK
Reading package lists...
Building dependency tree...
Reading state information...
Calculating upgrade...
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Installing playit ...
Playit.gg installed
Starting server...
Failed at initialized server
Charging status...
Failed to obtain IP
Obteniendo usuarios
Estas jugando a minecraft: forge en su version: 1.20.1
La versión de Forge es: 1.20.1-47.2.20
/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/1.20.1-47.2.20/unix_args.txt
Lista de usuarios:
Nombre: Leanne Graham, Email: Sincere@april.biz
Nombre: Ervin Howell, Email: Shanna@melissa.tv
Nombre: Clementine Bauch, Email: Nathan@yesenia.net
Nombre: Patricia Lebsack, Email: Julianne.OConner@kory.org
Nombre: Chelsey Dietrich, Email: Lucio_Hettinger@annie.ca
Nombre: Mrs. Dennis Schulist, Email: Karley_Dach@jasper.info
Nombre: Kurtis Weissnat, Email: Telly.Hoeger@billy.biz
Nombre: Nicholas Runolfsdottir V, Email: Sherwood@rosamond.me
Nombre: Glenn